In [1]:
import twitter
import os
from config import *
import time

# initialize api instance
twitter_api = twitter.Api(consumer_key=consumer_key,
                         consumer_secret=consumer_secret,
                         access_token_key=access_token_key,
                         access_token_secret=access_token_secret,
                        cache=None,tweet_mode='compat')

#print(consumer_key,consumer_secret,access_token_key,access_token_secret)

# test authentication
print(twitter_api.VerifyCredentials())
# print(twitter_api.GetStatus(126402758403305000))

{"created_at": "Wed Jul 20 02:34:27 +0000 2016", "description": "I am Al and I am passionate about technology.", "followers_count": 5, "friends_count": 52, "id": 755591669194514432, "id_str": "755591669194514432", "location": "Tustin, CA", "name": "Al", "profile_background_color": "000000", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_banner_url": "https://pbs.twimg.com/profile_banners/755591669194514432/1469041661", "profile_image_url": "http://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_link_color": "1B95E0", "profile_sidebar_border_color": "000000", "profile_sidebar_fill_color": "000000", "profile_text_color": "000000", "screen_name": "alknowstech", "status": {"created_at": "Mon Jul 31 21:38:46 +0000 2017", 

In [3]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100,
                                               lang='en',result_type='recent')
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None, "user":status.user.screen_name, "time":status.created_at,
                "search_term":search_keyword} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [4]:
# create blank testDataSet list
testDataSet = []

In [10]:
# test search function
search_term = input("Enter a search keyword: ")
testDataSet.extend(buildTestSet(search_term))

Enter a search keyword: mcdonalds
Fetched 100 tweets for the term mcdonalds


In [6]:
print(testDataSet[0:110])

[{'text': '@matvelloso Sounding like @nihilist_arbys today', 'label': None, 'user': 'richlilly', 'time': 'Fri Feb 21 18:15:59 +0000 2020', 'search_term': 'arbys'}, {'text': '@markjohnstonfm @Arbys I stopped eating there when they took the loaded potato bites off the menu. #arbys', 'label': None, 'user': 'Saskgirl316', 'time': 'Fri Feb 21 18:09:14 +0000 2020', 'search_term': 'arbys'}, {'text': "RT @miccaeli: and then steph meyer was straight up like 'and she is SOOOOO jealous of bella because edward loves BELLA and not HER and bell…", 'label': None, 'user': 'upsidedowndevil', 'time': 'Fri Feb 21 18:08:21 +0000 2020', 'search_term': 'arbys'}, {'text': "RT @miccaeli: and then steph meyer was straight up like 'and she is SOOOOO jealous of bella because edward loves BELLA and not HER and bell…", 'label': None, 'user': 'PotatoBugGirl', 'time': 'Fri Feb 21 18:02:43 +0000 2020', 'search_term': 'arbys'}, {'text': "RT @miccaeli: and then steph meyer was straight up like 'and she is SOOOOO jealou

# Use only to test twitter api

In [ ]:
tweets_fetched = twitter_api.GetSearch('disney', count = 100, lang='en')
[{"text":status.text, "label":None, "user":status.user.screen_name} for status in tweets_fetched]

# Start preprocessing of training data

In [11]:
import pandas as pd
import numpy as np

traindata140 = os.path.join(".","trainingandtestdata_sentiment140","training.csv")

In [12]:
#read the csv file command
train_df = pd.read_csv(traindata140, header=None, usecols=[0,5], names=['polarity of the tweet','text'], 
                       encoding="ISO-8859-1")
train_df.head()
# (0 = negative, 2 = neutral, 4 = positive)

,polarity of the tweet,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


# Use only to process the test dataset provided by Sentiment140

In [ ]:
testdata140 = os.path.join(".","trainingandtestdata_sentiment140","test.csv")
test_df = pd.read_csv(testdata140, header=None, usecols=[0,5],
                      names=['polarity of the tweet', 'text'],encoding="ISO-8859-1")
conditions_test = [
    (test_df['polarity of the tweet'] == 0),
    (test_df['polarity of the tweet'] == 2),
    (test_df['polarity of the tweet'] == 4)]
choices_test = ['negative', 'neutral', 'positive']
test_df['label'] = np.select(conditions_test, choices_test)
test_df.head()

test_ls = test_df.to_dict('records')

# Use to convert numeric value to Positive or Negative

In [13]:
train_df_negative = train_df.loc[train_df['polarity of the tweet']==0]
train_df_positive = train_df.loc[train_df['polarity of the tweet']==4]

In [14]:
# Pull evenly the amount of training set that will be used and combine them
frames = [train_df_negative.iloc[0:12500,:],train_df_positive.iloc[0:12500,:]]
result = pd.concat(frames)

In [15]:
conditions = [
    (result['polarity of the tweet'] == 0),
    (result['polarity of the tweet'] == 2),
    (result['polarity of the tweet'] == 4)]
choices = ['negative', 'neutral', 'positive']
result['label'] = np.select(conditions, choices)
result.head()

,polarity of the tweet,text,label
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative
1,0,is upset that he can't update his Facebook by ...,negative
2,0,@Kenichan I dived many times for the ball. Man...,negative
3,0,my whole body feels itchy and like its on fire,negative
4,0,"@nationwideclass no, it's not behaving at all....",negative


# Convert dataframe to dictionary to run analysis

In [16]:
train_ls = result.to_dict('records')

# Function to clean up the dataset

In [17]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk
# nltk.download('punkt')

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor = PreProcessTweets()

In [18]:
# Process Training Set
preprocessedTrainingSet = tweetProcessor.processTweets(train_ls)

In [19]:
# Process Test Set
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

# Function to process NLTK

In [22]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [23]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [24]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

# Execute only if model has to be retrained

In [ ]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

# SKLearnClassifier = SklearnClassifier(BernoulliNB()).train(trainingFeatures)
SKLearnClassifier = SklearnClassifier(SVC(), sparse=False).train(trainingFeatures)

# Compare the test set with the model

In [ ]:
SKLearnResultLabels = [SKLearnClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# Save the model to pickle for later use

In [ ]:
# Save Pickle
import pickle
save_classifier = open("sklearn_svc_train_25k.pickle","wb")
pickle.dump(SKLearnClassifier, save_classifier)
save_classifier.close()

# Use to load pickle

In [25]:
# Load Pickle
import pickle
classifier_f = open("sklearn_svc_train_25k.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

# Compare the test set with the loaded pickle

In [26]:
SKResultLabelsPickle = [classifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# Export results

In [27]:
# had to add space before tweet text to eliminate csv issue with tweets that begins with @
# list comprehension
test_list_text = [' '+test['text'] for test in testDataSet]
# convert twitter time output to real time output
test_list_time = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(test['time'],'%a %b %d %H:%M:%S +0000 %Y')) 
for test in testDataSet]
test_list_search_term = [test['search_term'] for test in testDataSet]

In [28]:
output_list = pd.DataFrame(
    {'Tweet': test_list_text,
     'Time': test_list_time,
     'Result': SKResultLabelsPickle,
     'Search_Term': test_list_search_term
    })

In [29]:
output_list['Score']=np.where((output_list.Result=='negative'), -1, 1)
output_list.head()

,Tweet,Time,Result,Search_Term,Score
0,@matvelloso Sounding like @nihilist_arbys today,2020-02-21 18:15:59,positive,arbys,1
1,@markjohnstonfm @Arbys I stopped eating there...,2020-02-21 18:09:14,positive,arbys,1
2,RT @miccaeli: and then steph meyer was straig...,2020-02-21 18:08:21,positive,arbys,1
3,RT @miccaeli: and then steph meyer was straig...,2020-02-21 18:02:43,positive,arbys,1
4,RT @miccaeli: and then steph meyer was straig...,2020-02-21 18:02:26,positive,arbys,1


In [30]:
output_list

,Tweet,Time,Result,Search_Term,Score
0,@matvelloso Sounding like @nihilist_arbys today,2020-02-21 18:15:59,positive,arbys,1
1,@markjohnstonfm @Arbys I stopped eating there...,2020-02-21 18:09:14,positive,arbys,1
2,RT @miccaeli: and then steph meyer was straig...,2020-02-21 18:08:21,positive,arbys,1
3,RT @miccaeli: and then steph meyer was straig...,2020-02-21 18:02:43,positive,arbys,1
4,RT @miccaeli: and then steph meyer was straig...,2020-02-21 18:02:26,positive,arbys,1
5,RT @miccaeli: and then steph meyer was straig...,2020-02-21 17:59:22,positive,arbys,1
6,RT @Arbys: @DOOM *Mick Gordon riffs intensify...,2020-02-21 17:55:33,positive,arbys,1
7,RT @miccaeli: and then steph meyer was straig...,2020-02-21 17:54:26,positive,arbys,1
8,Really @Arbys @ArbysCares this has gotten rid...,2020-02-21 17:53:03,negative,arbys,-1
9,when you eat at @Arbys and you feel that 'bub...,2020-02-21 17:52:46,negative,arbys,-1


# Output to CSV file

In [31]:
output_list.to_csv('./outputSKLearn_test.csv',header=True,
                              index=False,encoding="utf-8")